# Environment

In [1]:
# !pip install --upgrade pip

In [2]:
# !pip install matplotlib
# !pip install seaborn
# !pip install scikit-learn
# # !pip install keras
# # !pip install tensorflow_addons
# !pip install spacy
# !pip install gensim
# !pip install nltk
# !pip install psutil
# !pip install psycopg2
# !pip install pymorphy2

In [3]:
%load_ext autoreload
%autoreload 2

In [1]:
import matplotlib.pyplot as plt
%matplotlib inline

import seaborn as sns

c:\Users\Admin\.conda\envs\myenv310\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
c:\Users\Admin\.conda\envs\myenv310\lib\site-packages\numpy\.libs\libopenblas.EL2C6PLE4ZYW3ECEVIV3OXXGRN2NRFM2.gfortran-win_amd64.dll
c:\Users\Admin\.conda\envs\myenv310\lib\site-packages\numpy\.libs\libopenblas64__v0.3.23-246-g3d31191b-gcc_10_3_0.dll
c:\Users\Admin\.conda\envs\myenv310\lib\site-packages\numpy\.libs\libopenblas64__v0.3.23-gcc_10_3_0.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [ ]:
print(1)

In [2]:
# Basic
import os
import re
import sys
import glob
import gc
import pickle
import psutil
import psycopg2
import datetime
import numpy as np
import pandas as pd

from pathlib import Path
from pprint import pprint
from collections import Counter
from sklearn.preprocessing import LabelEncoder
from collections import defaultdict, namedtuple, deque, Counter
from typing import (List, Dict, Any, NoReturn, 
                    Tuple, Optional, Union)
from tqdm import tqdm
from datetime import datetime as dt

import multiprocessing
from sklearn.model_selection import train_test_split

import warnings
warnings.filterwarnings('ignore')

In [3]:
# W2V
import spacy
import gensim
import pymorphy2
from gensim.models import KeyedVectors
from gensim.models.phrases import Phrases
from nltk.stem.snowball import SnowballStemmer
from gensim.models.phrases import Phrases, npmi_scorer
from gensim.models import word2vec, keyedvectors


from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import (balanced_accuracy_score, accuracy_score, 
                             classification_report, confusion_matrix)

In [4]:
# System adjustments
pd.options.display.width = 2500
pd.options.display.max_rows = 999
pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', None)
pd.options.display.max_colwidth = 100

In [5]:
import logging
logging.basicConfig(format='%(asctime)s - %(message)s',
                    datefmt='%Y-%m-%d %H:%M:%S',
                    level=logging.INFO)

## Define paths

In [17]:
print(f"CPU count: {psutil.cpu_count()}")
print(f"CPU utilization: {psutil.cpu_percent()}%")
mem_stats = psutil.virtual_memory()
print(f"Memory total: {mem_stats.total / 1048576} MB.")
print(f"Memory available: {mem_stats.available / 1048576} MB.")
print(f"Memory used: {mem_stats.used / 1048576} MB.")

CPU count: 16
CPU utilization: 17.0%
Memory total: 32435.0546875 MB.
Memory available: 19402.9609375 MB.
Memory used: 13032.09375 MB.


In [42]:
BASE_DIR = Path('notebooks')
DATA_DIR = BASE_DIR / "data"
LISTS_DIR  = BASE_DIR / "lists"
# print("Avaliable incidents: ", [str(inc_id).split("/")[-1] for inc_id in DATA_DIR.glob("*")])

MODEL_DIR  = BASE_DIR / "models"
print("\nAvaliable models: ", [str(n.name) for n in MODEL_DIR.glob("*")])


Avaliable models:  []


In [19]:
LISTS_DIR

WindowsPath('notebooks/lists')

In [12]:
os.listdir(LISTS_DIR)

['intro.txt', 'NLTK_stopwords.txt']

In [23]:
# df = pd.read_csv('notebooks/data/parsed_reviews.csv')
df = pd.read_pickle('notebooks/data/Отзывы_инвестирование.pkl')

In [24]:
df = df.rename(columns={'Текст':'sentence'}).copy()
df.sample()

,Компания,url жалобы,Заголовок,Статус,sentence,Время,Оценка,Оценка выплат
1311,tcs,https://banki.ru/investment/responses/company/response/31135/,Реактивная реакция на проблему,Не засчитана,Быстренько и оперативно помогли закрыть дополнительный брокерский счёт который был открыт по ош...,31.08.2023,5,None


In [25]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3510 entries, 0 to 3509
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Компания       3510 non-null   object
 1   url жалобы     3510 non-null   object
 2   Заголовок      3510 non-null   object
 3   Статус         3244 non-null   object
 4   sentence       3510 non-null   object
 5   Время          3510 non-null   object
 6   Оценка         3244 non-null   object
 7   Оценка выплат  0 non-null      object
dtypes: object(8)
memory usage: 246.8+ KB


## Configure preprocessing

### Select only clients messages

### Preprocessing texts

In [36]:
def file_opener(filename: str) -> str:
    with open(filename, 'rt', encoding='utf-8-sig') as src:
        file = src.read()
    return "|".join([x for x in file.split('\n') if x])


class DataPreprocessorStemmer:

    text_features = ['sentence',]

    def __init__(self, intro_words_path: str = './lists'):
        self.__stemmer = SnowballStemmer("russian")
        self.__nlp = spacy.blank('ru')
        self.__intro_words = file_opener(os.path.join(intro_words_path, 'intro.txt'))
        self.__nltk_stopwords = file_opener(os.path.join(intro_words_path, 'NLTK_stopwords.txt')).split("|")

    def get_stopwords(self):
        """ Check intro-words list """
        return self.__nltk_stopwords

    def get_intro_words(self):
        """ Check intro-words list """
        return self.__intro_words

    def reformat_datetime(self, dt: datetime) -> str:
        """ Reformat datetime and convert to string """
        return dt.strftime("%Y-%m-%d %H-%M-%S")

    def process_text(self, texts: List[str]):
        """ Stem texts and return """
        texts = [re.sub(self.__intro_words, " ", text.strip(), flags=re.I) for text in texts]
        return [" ".join([self.__stemmer.stem(str(token).lower()) for token in doc
                          if (len(token.text) > 2 and token.is_alpha and ~token.is_stop
                              and (token.text not in self.__nltk_stopwords)
                              )])
                for doc in tqdm(self.__nlp.pipe(texts), total=len(texts))]


    def process(self, data: Union[List[Dict[str, Any]], pd.DataFrame],
                features_cols: Optional[List[str]] = None, copy: bool=True) -> Union[List[Dict[str, Any]], pd.DataFrame]:
        """
        Preprocess text for language modelling.
         - create joint columns (datetime, s_subject, subproduct, msg)
         - clean introduction words
         - tokenize and stemm texts
        """
        df_flg = False
        if type(data) == pd.DataFrame:
            df_flg = True
            data = data.to_dict('records')
        print("[INFO] Text processing...")
        if not features_cols:
            features_cols = self.text_features

        for col_name in features_cols:
            print(f"[INFO] Process '{col_name}' column...")
            data_col_name = self.process_text([req.get(col_name, "") if req.get(col_name, "") else ""
                                               for req in data])
            if not copy:
                _ = [req.update({col_name: val}) for req, val in zip(data, data_col_name)]
            else:
                _ = [req.update({col_name + "_proc": val}) for req, val in zip(data, data_col_name)]

        print("[INFO] Text finished.")
        return pd.DataFrame(data) if df_flg else data

In [35]:



def file_opener(filename: str) -> str:
    with open(filename, 'rt', encoding='utf-8-sig') as src:
        file = src.read()
    return "|".join([x for x in file.split('\n') if x])


class DataPreprocessorLemmatizer:

    text_features = ['msg']
    stopgrams = [
             'CONJ',   # союз
             'PRCL',   # частица
             'PRED',   # предикатив
             'NPRO',   # местоимение-сущ.
             'INTJ',   # междометие
             'Erro',   # ошибка
             'Dist',   # искажение
             'Ques',   # вопросительное слово
             'Dmns',   # указательное слово
             'Prnt'   # вводное слово
            ]

    def __init__(self, multipocess: bool, num_processors: int=16, chunksize: int=100, 
                intro_words_path: str = './lists'):
        # Language parsers
        self.__morph = pymorphy2.MorphAnalyzer()
        self.__nlp = spacy.blank('ru')
        # Multiprocessing params
        self.__multipocess = multipocess
        self.__num_processors = num_processors
        self.__chunksize = chunksize
        # Cleaning utils
        self.__intro_words = file_opener(os.path.join(intro_words_path, 'intro.txt'))
        self.__nltk_stopwords = file_opener(os.path.join(intro_words_path, 'NLTK_stopwords.txt')).split("|")

    def get_stopwords(self):
        """ 
        Check intro-words list.
        """
        return self.__nltk_stopwords

    def get_intro_words(self):
        """ 
        Check intro-words list.
        """
        return self.__intro_words
    
    def get_analyzer(self):
        """
        Allow to access to Pymorphy Analyzer instance.
        """
        return self.__morph

    
    def _process_text(self, text: str):
        """ 
        Process single text and return list of tokens.
        """
        if pd.isna(text):
            return []
       # Pre-processing part 
        text = [str(token).lower()
                for token in self.__nlp.make_doc(text)
                if (token and token.is_alpha and len(str(token.text)) > 2 and ~token.is_stop)]
        # Processing part
        clean_text = []
        for token in text:
            token = self.__morph.parse(str(token).lower())[0]
            if ((token.normal_form not in self.__nltk_stopwords) 
                and (token.normal_form not in self.__intro_words)
                and all([tag not in token.tag for tag in self.stopgrams])):
                clean_text.append(token.normal_form)
        return clean_text
    

    def process_texts(self, texts: List[str]):
        """ 
        Process list of texts and return list of lists of tokens.
        """
        if self.__multipocess:
            with multiprocessing.Pool(self.__num_processors) as pool:
                processed_texts = list(tqdm(pool.imap(self._process_text, texts, 
                                                      chunksize=self.__chunksize), 
                                                     total=len(texts)))
            logging.info(f"Texts processed.")
            return processed_texts
        else:
            return [self._process_text(text) for text in tqdm(texts)]
        


    def process(self, data: pd.DataFrame,
                features_cols: Optional[List[str]] = None, copy: bool=True) -> pd.DataFrame:
        """
        Preprocess text for language modelling.
         - clean introduction words, numbers and small prefixes;
         - tokenize and lemmatize texts;
        """
        logging.info("Text processing started.")
        if not features_cols:
            features_cols = self.text_features

        for col_name in features_cols:
            logging.info(f"Processing '{col_name}' column...")
            data_processed = self.process_texts(data[col_name].fillna("").to_list())
            if not copy:
                data[col_name] = data_processed
            else:
                data[col_name + "_proc"] = data_processed

        logging.info("Text preprocessing finished.")
        return data

In [37]:
processor = DataPreprocessorLemmatizer(multipocess=False,
                                       num_processors=4, chunksize=400,
                                       intro_words_path=str(LISTS_DIR))

2023-09-17 20:39:09 - Loading dictionaries from c:\Users\Admin\.conda\envs\myenv310\lib\site-packages\pymorphy2_dicts_ru\data
2023-09-17 20:39:09 - format: 2.4, revision: 417127, updated: 2020-10-11T15:05:51.070345


In [38]:
%%time
df1 = processor.process(data=df, features_cols=['sentence'])

2023-09-17 20:39:11 - Text processing started.
2023-09-17 20:39:11 - Processing 'sentence' column...
100%|██████████| 3510/3510 [01:23<00:00, 41.87it/s]
2023-09-17 20:40:35 - Text preprocessing finished.


CPU times: total: 1min 21s
Wall time: 1min 23s


In [32]:
df1

,Компания,url жалобы,Заголовок,Статус,sentence,Время,Оценка,Оценка выплат,sentence_proc
0,alfa-direkt,https://banki.ru/investment/responses/company/response/31299/,Отзыв,Проверяется,"«У меня был исключительный опыт работы с Альфа-Банком! 23 июня, около 12:00, я связался, чтобы у...",14.09.2023,5,None,"[исключительный, опыт, работа, альфа, банк, июнь, около, связаться, наиболее, выгодный, вариант,..."
1,alfa-direkt,https://banki.ru/investment/responses/company/response/31298/,Хороший опыт!,Проверяется,"«У меня был отличный опыт работы с Альфа-Банком! Я связался с ними 23 марта около 12:00, чтобы п...",14.09.2023,5,None,"[опыт, работа, альфа, банк, связаться, март, около, получить, консультация, наиболее, выгодный, ..."
2,alfa-direkt,https://banki.ru/investment/responses/company/response/31286/,Закрытие позиций брокером без поручения клиента,Проверяется,"Добрый деньНомер счета 1160988-000На каком основании вчера, 13 сентября 10:47 закрыта короткая п...",14.09.2023,1,None,"[деньномер, счёт, основание, вчера, сентябрь, закрытый, короткий, позиция, gdr, бумага, торговат..."
3,alfa-direkt,https://banki.ru/investment/responses/company/response/31214/,Отвратительное качество it-сервисов,Проверяется,"Мой брокерский счёт перевели в ""альфу"" из ВТБ. Потом альфа тоже попала под санкции, и, когда я в...",08.09.2023,1,None,"[брокерский, счёт, перевести, альф, втб, альфа, попасть, санкция, вспомнить, вывести, деньга, ча..."
4,alfa-direkt,https://banki.ru/investment/responses/company/response/31210/,Компания списывает мои деньги со счета,Проверяется,"На написание отзыва меня сподвигло не решение вопроса более 3 недель,не уважение к моей проблеме...",07.09.2023,1,None,"[написание, отзыв, сподвигнуть, решение, вопрос, неделя, уважение, проблема, уходить, деньга, сч..."
...,...,...,...,...,...,...,...,...,...
3505,vtb,https://banki.ru/investment/responses/company/response/26476/,ВТБ брокер: ни в коем случае!,,"Завёл деньги в феврале 2021, купил доллары, акции, торгуемые в Nasdaq за доллары. Очередные диви...",16.12.2021,1,None,"[завести, деньга, февраль, купить, доллар, акция, торговать, nasdaq, доллар, очередной, дивиденд..."
3506,vtb,https://banki.ru/investment/responses/company/response/26473/,Худшая служба поддержки в моей жизни,,"6 декабря я пришёл к ним в отделение открывать брокерские счёта. Сказал, чтобы мне открыли один ...",15.12.2021,1,None,"[декабрь, прийти, отделение, открывать, брокерский, счёт, открыть, иис, счёт, обыкновенный, брок..."
3507,vtb,https://banki.ru/investment/responses/company/response/26471/,"Дорогая редакция, я плакал!",,"Я клиент брокера с 2019 года, номер договора 1021 BA, также у меня открыт ранее другой брокерски...",14.12.2021,1,None,"[клиент, брокер, номер, договор, открыть, ранее, брокерский, счёт, открытие, решить, закрыть, сч..."
3508,vtb,https://banki.ru/investment/responses/company/response/26468/,Отсутствие двухфакторной авторизации в личном кабинете ВТБ Брокера,Не засчитана,Здравствуйте!Прошу доработать авторизацию и сделать ее двухфакторной (по логину-паролю + СМС с к...,13.12.2021,1,None,"[доработать, авторизация, двухфакторный, логин, пароль, смс, код, кабинет, втб, инвестиция, серь..."


In [39]:
df1.to_pickle('notebooks/data/data_preproc_all.pkl')

In [40]:
Counter(df1.explode('sentence_proc').sentence_proc.values).most_common()

[('счёт', 4216),
 ('брокер', 3721),
 ('акция', 2827),
 ('инвестиция', 2177),
 ('вопрос', 2158),
 ('поддержка', 1994),
 ('приложение', 1944),
 ('брокерский', 1923),
 ('клиент', 1893),
 ('свой', 1888),
 ('иис', 1879),
 ('бумага', 1697),
 ('сотрудник', 1649),
 ('деньга', 1510),
 ('ответ', 1419),
 ('проблема', 1402),
 ('перевод', 1268),
 ('получить', 1241),
 ('время', 1216),
 ('чат', 1213),
 ('решить', 1180),
 ('офис', 956),
 ('обращение', 954),
 ('актив', 952),
 ('перевести', 923),
 ('заявка', 877),
 ('хотеть', 875),
 ('открыть', 867),
 ('средство', 842),
 ('ситуация', 841),
 ('комиссия', 836),
 ('рубль', 830),
 ('работать', 821),
 ('месяц', 819),
 ('большой', 818),
 ('написать', 782),
 ('информация', 762),
 ('работа', 758),
 ('сумма', 757),
 ('срок', 747),
 ('ответить', 711),
 ('закрыть', 701),
 ('поручение', 697),
 ('продать', 682),
 ('ценный', 661),
 ('писать', 656),
 ('решение', 655),
 ('документ', 650),
 ('должный', 637),
 ('помочь', 634),
 ('обратиться', 619),
 ('менеджер', 611),
 (